In [ ]:
!py -3.10 -m pip install xai-sdk # 3.10.11 버전 세팅 

In [ ]:
pip install python-dotenv 

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일 경로 지정, 이상은 노트북 로컬 화경에서의 grok AI키
dotenv_path = r"C:\Users\EL06\Desktop\project_1\env"
load_dotenv(dotenv_path)

# 환경 변수 가져오기
api_key = os.getenv("XAI_API_KEY")
print("키 찾음:", api_key[:10] + "..." if api_key else "없음!")

키 찾음: xai-1WfA3f...


In [20]:
from xai_sdk import Client
from xai_sdk.chat import user, system
from IPython.display import Markdown, display
import os
import json 

# ----------------- 1. 환경 설정 및 클라이언트 초기화 -----------------
# 이전 단계에서 api_key가 os.environ에 성공적으로 로드되었다고 가정합니다.
# Grok 클라이언트 초기화
try:
    # XAI_API_KEY 환경 변수가 로드되어 있어야 합니다.
    client = Client(api_key=os.getenv("XAI_API_KEY")) 
except Exception as e:
    print("API 클라이언트 초기화 실패. XAI_API_KEY를 확인하세요.")
    # 실제 환경에서는 이 부분에서 종료하거나 키를 입력받아야 합니다.


# ----------------- 2. 핵심: 상세 시스템 프롬프트 정의 -----------------
# 당신의 역할과 문장 구성 규칙, 톤을 명확히 지시합니다.
SYSTEM_PROMPT = """
당신은 이미지 기반 심리검사 결과를 해석하고 사용자에게 전달하는 **전문 컨설턴트**입니다.

## 🔑 핵심 전달 원칙 및 형식:
1.  **톤**: 사용자가 **절대 상처받지 않도록** 매우 따뜻하고, 공감하며, 긍정적인 방향으로 문장을 구성해야 합니다. 문체는 유하고 희망적이어야 합니다.
2.  **구조**:
    * **첫 줄**: 제공된 결과값 중 **가장 중요하고 핵심적인 요소**에 한정하여 **정확히 두 문장으로 압축 요약**합니다.
    * **그 아래아래 줄 (두 줄 공백)**: 상세 해석 내용을 **정확히 다섯 줄로 정리하여 요약**합니다.
3.  **길이 조정 (가장 중요)**:
    * 입력 데이터가 짧더라도 **할루시네이션(새로운 사실 생성) 없이** 기존 데이터를 깊이 있게 분석하고, 잠재력에 대한 긍정적인 해석을 덧붙여 의미를 확장함으로써 **반드시 다섯 줄을 완성해야 합니다.** (문장의 일관성과 타당성을 유지하면서 분량을 늘릴 것)
"""

# ----------------- 3. 샘플 입력 데이터 정의 -----------------
# 실제 서버에서 JSON으로 받을 데이터를 가정하여 예시를 만듭니다.
# (예시: 내용이 짧은 경우를 가정하여 5줄 확장 규칙 테스트)
sample_result_data = {
    "검사_결과_유형": "내면 탐구형",
    "핵심_점수": {"이름": "창의적 사고", "점수": 92, "해석": "독창적인 시각과 뛰어난 예술적 감각이 발달되어 있습니다."},
    "보조_조언": "일상의 작은 변화를 통해 창의성을 발휘할 기회를 늘려보세요."
}

# ----------------- 4. Grok 채팅 세션 생성 및 요청 -----------------
# 온도를 0.8로 설정하여 따뜻하고 유연한 문체를 유도합니다.
TEMP = 0.8 
chat_session = client.chat.create(model='grok-4', temperature=TEMP)

# 4-1. 시스템 프롬프트 적용
chat_session.append(system(SYSTEM_PROMPT))

# 4-2. 사용자 데이터 입력 메시지 생성
# JSON 데이터를 문자열로 변환하여 Grok에게 전달합니다.
user_message = f"""
다음은 이미지 심리검사 결과 데이터입니다:

{json.dumps(sample_result_data, indent=4, ensure_ascii=False)}

위 데이터를 분석하여, 당신의 전달 원칙과 형식(2줄 요약 + 5줄 상세)에 맞춰 최종 해석 문장을 생성해 주세요.
"""
chat_session.append(user(user_message))

# ----------------- 5. 응답 요청 및 출력 -----------------
print(f"## 🤖 Grok-4 심리검사 해석 결과 요청 (Temperature: {TEMP})")
print("-" * 50)

# API 호출 및 응답 출력
try:
    response = chat_session.sample()
    # Markdown으로 출력하여 가독성을 높입니다.
    display(Markdown(response.content))
except Exception as e:
    print(f"Grok API 호출 중 오류가 발생했습니다: {e}")

## 🤖 Grok-4 심리검사 해석 결과 요청 (Temperature: 0.8)
--------------------------------------------------


KeyboardInterrupt: 